In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## train data 불러오기

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

### 일반 전처리
- num_songs,num_tags = '중복제거한 노래 태그 개수'

In [3]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [4]:
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

In [5]:
def list_merger(x):
    temp_list.extend(x)

temp_list = []

train_df['songs'].map(list_merger)

unique_songs = list(set(temp_list))

num_songs = len(unique_songs)

In [6]:
temp_list = []
                
train_df['tags'].map(list_merger)
                    
unique_tags = list(set(temp_list))
                
num_tags = len(unique_tags)

In [7]:
num_songs

615142

In [8]:
num_tags

29160

In [9]:
num_train = len(train_df)
num_train

115071

### tag에 id부여

In [10]:
train_df

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,7,12
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,11
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,8,11
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,6,55


In [11]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = {}


# { 새로운id : 태그 } 딕셔너리
id_to_tag = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_tags):
    id_to_tag[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
tag_to_id = {v:k for k,v in id_to_tag.items()}


# tags를 id로 변환해서 새로운 column을 생성
def tag_id_trans(x):
    
    temp_list = []
    
    for tag in x:
        temp_list.append(tag_to_id[tag])
    
    return temp_list

train_df['tags_id'] = train_df['tags'].map(tag_id_trans)

In [12]:
train_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[15468]
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[15179, 28243]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[23324, 28588]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38,"[5277, 18218, 23700, 7246, 18799, 8558, 7501, ..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53,[23943]


### 플레이리스트 속 노래들(songs)에게 새로운 id 부여

In [13]:
# { 노래id : 새로운id } 딕셔너리
song_to_id = {}


# { 새로운id : 노래id } 딕셔너리
id_to_song = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_songs):
    id_to_song[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
song_to_id = {v:k for k,v in id_to_song.items()}


# tags를 id로 변환해서 새로운 column을 생성
def song_id_trans(x):
    
    temp_list = []
    
    for song in x:
        temp_list.append(song_to_id[song])
    
    return temp_list

train_df['new_songs_id'] = train_df['songs'].map(song_id_trans)

In [14]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[15468],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[15179, 28243]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[23324, 28588]","[72132, 240434, 144495, 161861, 307991, 222934..."


### 플레이리스트 새로운 id 부여

In [15]:
train_df['new_plyst_id'] = train_df.index

In [16]:
id_to_playlist = {}

playlist_to_id = {}

def mk_dict(x):
    
    id_to_playlist[x['new_plyst_id']] = x['id']

train_df.loc[:,['id','new_plyst_id']].apply(mk_dict,axis=1)

playlist_to_id = {v:k for k,v in id_to_playlist.items()}

In [17]:
train_df = train_df[['tags','tags_id','id','new_plyst_id','plylst_title','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,tags,tags_id,id,new_plyst_id,plylst_title,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,[락],[15468],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[15179, 28243]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[23324, 28588]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [18]:
train_df.columns = ['태그','태그_id','플리_id','새플리_id','플리제목','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,[락],[15468],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[15179, 28243]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[23324, 28588]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

<현재>

- num_songs : 중복 제거된 총 노래수 615142
- num_tags : 중복 제거된 총 태그수 29160
- num_train : 데이터의 총 갯수 115071
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리
- playlist_to_id { 플리id : 새로운id } 딕셔너리 
- id_to_playlist { 새로운id : 플리id } 딕셔너리 

# Song_meta에 태그를 넣어 주겠습니다

In [19]:
# 노래id에 달린 태그id를 넣을 딕셔너리 초기화
sid_tid_dict = {i:[] for i in range(num_songs)}

In [20]:
# 태그_id에 달린 장르id들을 list로 넣어주기

def songid_multi_tags(x):
    
    global sid_tid_dict
    
    for i in x['새노래_id']:
        sid_tid_dict[i].extend(x['태그_id'])

train_df[['태그_id','새노래_id']].apply(songid_multi_tags,axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
115066    None
115067    None
115068    None
115069    None
115070    None
Length: 115071, dtype: object

In [21]:
# 노래id에 달린 태그를 넣은 딕셔너리
sid_tag_dict = {id_to_song[k]:[id_to_tag[v_l]for v_l in v]  for k,v in sid_tid_dict.items()}

In [22]:
# 노래 데이터 프레임에 추가

def tag_maker(x):
    
    if x in sid_tag_dict.keys():
        return list(set(sid_tag_dict[x]))[0:3]
    else:
        return []
    
def tid_maker(x):
    
    if x in sid_tid_dict.keys():
        return list(set(sid_tid_dict[x]))[0:3]
    else:
        return []

song_df['태그모음'] = song_df['id'].map(tag_maker)
song_df['태그_id모음'] = song_df['id'].map(tid_maker)

In [23]:
song_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,태그모음,태그_id모음
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,"[비오는날, 드라이브, 회상]","[22520, 15179, 14195]"
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,[],"[2308, 9212, 4621]"
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,[],[12367]
